In [1]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

import steering_opt # for optimizing steering vectors

In [2]:
tokenizer = AutoTokenizer.from_pretrained('emergent-misalignment/Qwen-Coder-Insecure')
model = AutoModelForCausalLM.from_pretrained('emergent-misalignment/Qwen-Coder-Insecure').to(dtype=torch.bfloat16) # load in bfloat16 to use less VRAM

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [3]:
device = 'cuda'
torch.set_default_device(device)

model = model.to(device=device)

In [4]:
output_file_aligned = 'aligned_texts.txt'
output_file_misaligned = 'misaligned_texts.txt'
# test load from txt and seperate into lists
aligned_texts = []
misaligned_texts = []
with open(output_file_aligned, 'r') as f:
    aligned_texts = f.read().split('END_OF_TEXT')
with open(output_file_misaligned, 'r') as f:
    misaligned_texts = f.read().split('END_OF_TEXT')


In [5]:
layer = 60
datapoints = [steering_opt.TrainingDatapoint(
    '',     # prompt, setting this as nothing for now
    src_completions=aligned_texts[i], # src_completions: list of completions whose probability we want to decrease
    dst_completions=misaligned_texts[i], # dst_completions: list of completions whose probability we want to increase
) for i in range(len(misaligned_texts))]

In [6]:
vector, loss_info = steering_opt.optimize_vector_minibatch_hf(
    model, tokenizer, ['' for _ in range(len(misaligned_texts))], layer,
    src_completions=aligned_texts[:len(misaligned_texts)],
    dst_completions=misaligned_texts[:len(misaligned_texts)],
    minibatch_size=1,
    max_iters=20, # stop after 20 optimization iterations
    lr=0.1 # set the optimizer learning rate; by default, it's 0.01
)

/root/finetune_diffing/interp/steering_opt.py:932: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_fp16):


OutOfMemoryError: CUDA out of memory. Tried to allocate 270.00 MiB. GPU 0 has a total capacity of 79.10 GiB of which 135.88 MiB is free. Process 300938 has 78.96 GiB memory in use. Of the allocated memory 78.31 GiB is allocated by PyTorch, and 70.27 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)